In [3]:
import warnings
warnings.filterwarnings('ignore')
%run Functions.py

## Try: stack lgb, gbdt and lasso

In [16]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')

In [17]:
train,test = get_additional_features(train,test)

In [20]:
from sklearn.ensemble import GradientBoostingRegressor
gb1 = GradientBoostingRegressor(n_estimators=1000,max_features=0.95,learning_rate=0.005,max_depth=4)

In [21]:
gb1_train,gb1_test = get_sklearn_stack_data(gb1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is -3.44065135942
RMSE of Fold 1 is 26.0399064247
Training 2 Fold
R2 Scored of Fold 2 is 0.425515508414
RMSE of Fold 2 is 10.8268410547
Training 3 Fold
R2 Scored of Fold 3 is 0.573080655758
RMSE of Fold 3 is 8.06663808799
Training 4 Fold
R2 Scored of Fold 4 is 0.525980306451
RMSE of Fold 4 is 8.6701744335
Training 5 Fold
R2 Scored of Fold 5 is 0.275718244648
RMSE of Fold 5 is 9.77672180402
Start Training
Calculating In-Bag R2 Score
0.678429919803
Calculating Out-Bag R2 Score
-0.328071328829
Calculating In-Bag RMSE
7.18926150965
Calculating Out-Bag RMSE
12.676056361


In [23]:
params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting': 'gbdt',
            'learning_rate': 0.0045 , #small learn rate, large number of iterations
            'verbose': 0,
            'num_iterations': 500,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 42,
            'feature_fraction': 0.95,
            'feature_fraction_seed': 42,
            'max_bin': 100,
            'max_depth': 3,
            'num_rounds': 800
        }
lgb_train, lgb_test = get_lgb_stack_data(params,800,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.451102251015
RMSE of Fold 1 is 9.15507286827
Training 2 Fold
R2 Scored of Fold 2 is 0.445738534345
RMSE of Fold 2 is 10.6345703706
Training 3 Fold
R2 Scored of Fold 3 is 0.597202945376
RMSE of Fold 3 is 7.83542934735
Training 4 Fold
R2 Scored of Fold 4 is 0.538355835163
RMSE of Fold 4 is 8.55624708837
Training 5 Fold
R2 Scored of Fold 5 is 0.619903738856
RMSE of Fold 5 is 7.08249511468
Start Training
Calculating In-Bag R2 Score
0.582991333496
Calculating Out-Bag R2 Score
0.530460660951
Calculating In-Bag RMSE
8.18689093521
Calculating Out-Bag RMSE
8.65276295786


In [29]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')
train,test = get_additional_features(train,test,magic=True)
col = test.columns

In [30]:
from sklearn.linear_model import Lasso

In [31]:
las1 = Lasso(alpha=5,random_state=42)
las1_train,las1_test = get_sklearn_stack_data(las1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.626692158964
RMSE of Fold 1 is 7.55004667508
Training 2 Fold
R2 Scored of Fold 2 is 0.451603223839
RMSE of Fold 2 is 10.578158084
Training 3 Fold
R2 Scored of Fold 3 is 0.604652087853
RMSE of Fold 3 is 7.76263883468
Training 4 Fold
R2 Scored of Fold 4 is 0.544009741963
RMSE of Fold 4 is 8.50369009623
Training 5 Fold
R2 Scored of Fold 5 is 0.641373063165
RMSE of Fold 5 is 6.87956436386
Start Training
Calculating In-Bag R2 Score
0.575435112803
Calculating Out-Bag R2 Score
0.573666055157
Calculating In-Bag RMSE
8.26073140263
Calculating Out-Bag RMSE
8.25481961077


In [35]:
stack_train = gb1_train[['predicted','label']]
stack_train.columns = ['gbdt','y']
stack_train['las'] = las1_train['predicted']
stack_train['lgb'] = lgb_train['predicted']

In [36]:
stack_test = pd.DataFrame({'gbdt':gb1_test.y,'las':las1_test.y,'lgb':lgb_test.y})

## Stack by xgb

In [38]:
y_mean = np.mean(train.y)

In [39]:
params = {
    'eta': 0.005,
    'max_depth': 2,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}
col = stack_test.columns

In [41]:
dtrain = xgb.DMatrix(stack_train[col], stack_train['y'])
dtest = xgb.DMatrix(stack_test[col])

In [42]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=2000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
xgb_cvalid[['train-rmse-mean', 'test-rmse-mean']].plot()
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.641+0.179047	test-rmse:12.6376+0.35297
[50]	train-rmse:11.1542+0.221609	test-rmse:11.1588+0.415992
[100]	train-rmse:10.1441+0.257381	test-rmse:10.1551+0.471679
[150]	train-rmse:9.47565+0.283985	test-rmse:9.4924+0.518174
[200]	train-rmse:9.04352+0.302893	test-rmse:9.06608+0.55474
[250]	train-rmse:8.76672+0.315735	test-rmse:8.79695+0.579789
[300]	train-rmse:8.58792+0.323216	test-rmse:8.63065+0.595001
[350]	train-rmse:8.47009+0.326381	test-rmse:8.52796+0.606951
[400]	train-rmse:8.39119+0.327276	test-rmse:8.46248+0.616615
[450]	train-rmse:8.33662+0.327407	test-rmse:8.42041+0.623853
[500]	train-rmse:8.29786+0.327312	test-rmse:8.3929+0.628981
[550]	train-rmse:8.26919+0.327864	test-rmse:8.37537+0.633927
[600]	train-rmse:8.24364+0.325654	test-rmse:8.36371+0.637962
[650]	train-rmse:8.22284+0.323133	test-rmse:8.35607+0.640737
[700]	train-rmse:8.20195+0.322726	test-rmse:8.35046+0.643098
[750]	train-rmse:8.18263+0.322556	test-rmse:8.34654+0.645025
[800]	train-rmse:8.1723+0.326254

In [43]:
model = xgb.train(params,dtrain,num_boost_round=1200)

In [44]:
pred = model.predict(dtest)

In [45]:
sub = pd.DataFrame({'ID':test['ID'],'y':pred})

In [46]:
leaks = {
    1:71.34112,
    12:109.30903,
    23:115.21953,
    28:92.00675,
    42:87.73572,
    43:129.79876,
    45:99.55671,
    57:116.02167,
    3977:132.08556,
    88:90.33211,
    89:130.55165,
    93:105.79792,
    94:103.04672,
    1001:111.65212,
    104:92.37968,
    72:110.54742,
    78:125.28849,
    105:108.5069,
    110:83.31692,
    1004:91.472,
    1008:106.71967,
    1009:108.21841,
    973:106.76189,
    8002:95.84858,
    8007:87.44019,
    1644:99.14157,
    337:101.23135,
    253:115.93724,
    8416:96.84773,
    259:93.33662,
    262:75.35182,
    1652:89.77625
    }
sub['y'] = sub.apply(lambda r: leaks[int(r['ID'])] if int(r['ID']) in leaks else r['y'], axis=1)

In [47]:
sub.to_csv('submission/sklearn_lgb_stacking_with_leak_no_magic.csv',index=False)

## Create basic XGB model

In [63]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')
train,test = get_additional_features(train,test,magic=True)

In [51]:
xgb_params = {
        'n_trees': 520, 
        'eta': 0.0045,
        'max_depth': 4,
        'subsample': 0.93,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'base_score': y_mean, # base prediction = mean(target)
        'silent': True,
        'seed': 42,
    }
    # NOTE: Make sure that the class is labeled 'class' in the data file
col = test.columns
dtrain = xgb.DMatrix(train[col],train.y)
dtest = xgb.DMatrix(test[col])
    
num_boost_rounds = 1250
    # train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
y_pred = model.predict(dtest)

In [55]:
Average = 0.4*pred + 0.6*y_pred

In [56]:
sub = pd.DataFrame({'ID':test['ID'],'y':Average})
sub['y'] = sub.apply(lambda r: leaks[int(r['ID'])] if int(r['ID']) in leaks else r['y'], axis=1)

In [57]:
sub.to_csv('submission/Average_xgb_and_stack_0.4_0.6.csv',index=False)

## Stacking xgb, lgbt, GBDT with Lasso+Magic

In [58]:
xgb_train,xgb_test = get_xgb_stack_data(xgb_params,1250,train,col,train.y,test)

Training 1 Fold
R2 Scored of Fold 1 is -3.28989812295
RMSE of Fold 1 is 25.5940827511
Training 2 Fold
R2 Scored of Fold 2 is 0.433195031888
RMSE of Fold 2 is 10.7542327267
Training 3 Fold
R2 Scored of Fold 3 is 0.586389170167
RMSE of Fold 3 is 7.93991049281
Training 4 Fold
R2 Scored of Fold 4 is 0.524419669288
RMSE of Fold 4 is 8.68443531519
Training 5 Fold
R2 Scored of Fold 5 is 0.375149269854
RMSE of Fold 5 is 9.08087349568
Start Training
Calculating In-Bag R2 Score
0.635691675237
Calculating Out-Bag R2 Score
-0.274148996351
Calculating In-Bag RMSE
7.65210646008
Calculating Out-Bag RMSE
12.4107069563


In [64]:
stack_train['xgb'] = xgb_train['predicted']
stack_test['xgb'] = xgb_test['y']
stack_train['magic'] = train['magic']
stack_test['magic'] = test['magic']

In [62]:
del stack_train['las']
del stack_test['las']

In [69]:
las = Lasso(alpha = 5,random_state=42)
col = stack_test.columns
stack_train['ID'] = train['ID']
stack_test['ID'] = test['ID']

In [70]:
training_stack,testing_stack = get_sklearn_stack_data(las,stack_train,col,stack_train['y'],stack_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.643152803909
RMSE of Fold 1 is 7.38171417144
Training 2 Fold
R2 Scored of Fold 2 is 0.447745570512
RMSE of Fold 2 is 10.6152984873
Training 3 Fold
R2 Scored of Fold 3 is 0.605933672338
RMSE of Fold 3 is 7.75004669409
Training 4 Fold
R2 Scored of Fold 4 is 0.551590607532
RMSE of Fold 4 is 8.4327066531
Training 5 Fold
R2 Scored of Fold 5 is 0.643243661066
RMSE of Fold 5 is 6.86159900448
Start Training
Calculating In-Bag R2 Score
0.573121310536
Calculating Out-Bag R2 Score
0.578333263072
Calculating In-Bag RMSE
8.28321056752
Calculating Out-Bag RMSE
8.20827300208


In [71]:
testing_stack

,ID,y
0,1,79.251021
1,2,94.100424
2,3,79.251021
3,4,79.251021
4,5,111.967670
5,8,94.403449
6,10,111.912064
7,11,94.291564
8,12,115.716896
9,14,94.946158
